In [ ]:
%pip install faiss-cpu

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os

In [ ]:
%mamba install psycopg2

In [ ]:
from langchain_community.utilities import SQLDatabase

POSTGRESQL_URI = os.getenv('POSTGRESQL_URI')
db = SQLDatabase.from_uri(POSTGRESQL_URI)
print(db.dialect)
print(db.get_usable_table_names())

In [ ]:
db.run("SELECT * FROM table_name LIMIT 10;")

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_core.prompts import PromptTemplate
template = 'You are a PostgreSQL expert. Given an input question, create a syntactically correct PostgreSQL query to run.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\nPay attention to use CURRENT_DATE function to get the current date, if the question involves "today".\n\nOnly use the following tables:\n{table_info}\n\nRespond only with a SQL query without additional formatting.\n\nHere is an example:\n\nQuestion: How many authors are there\nYour Answer: SELECT COUNT("id") AS "author_count" FROM authors;\n\nQuestion: {input}\nYour Answer: '

prompt = PromptTemplate.from_template(template)

In [ ]:
from langchain.chains import create_sql_query_chain

table_names_to_use=[]

chain = create_sql_query_chain(llm, db, prompt=prompt)
print(chain)

In [ ]:
chain.get_prompts()

In [ ]:
db.run(response)

In [ ]:
response = chain.invoke({
  "question": "What is ...",
  "table_names_to_use": table_names_to_use
})
response

In [ ]:
db.run(response)

In [ ]:
generated_query = chain.invoke(
  {
    "question": "How many ...",
    "table_names_to_use": table_names_to_use
  }
)
generated_query

In [ ]:
generated_query = chain.invoke(
  {
    "question": "What is the ...",
    "table_names_to_use": table_names_to_use
  }
)
generated_query

In [ ]:
sql_result = db.run(generated_query)

In [ ]:
sql_result

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

final_chain = (
    answer_prompt
    | llm
    | StrOutputParser()
)

final_chain.invoke({
    "question": "How many ...",
    "query": generated_query,
    "result": sql_result
})

In [ ]:
final_chain.invoke({
    "question": "What is the ...",
    "query": generated_query,
    "result": sql_result
})

In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()
tools

In [ ]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

In [ ]:
%mamba install --quiet langgraph

In [ ]:
%pip install --upgrade --quiet langchain_core

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)

In [ ]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="How many ...?")]}
):
    print(s)
    print("----")

In [ ]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Describe table table_name")]}
):
    print(s)
    print("----")

In [ ]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="What is the ...?")]}
):
    print(s)
    print("----")

In [ ]:
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    return list(set(res))


list_of_stuff = query_as_list(db, "SELECT column_name FROM table_name")
list_of_stuff[:5]

In [ ]:
len(list_of_stuff)

In [ ]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vector_db = FAISS.from_texts(list_of_stuff, OpenAIEmbeddings())
retriever = vector_db.as_retriever(search_kwargs={"k": 5})
description = """Use to look up values to filter on. Input is an approximate spelling of the proper noun, output is \
valid proper nouns. Use the noun most similar to the search."""
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

In [ ]:
print(retriever_tool.invoke("similar term"))

In [ ]:
system = """You are an agent designed to interact with a PostgreSQL database.
Given an input question, create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

You have access to the following tables: {table_names}

If you need to filter on a proper noun, you must ALWAYS first look up the filter value using the "search_proper_nouns" tool!
Do not try to guess at the proper name - use this function to find similar ones.""".format(
    table_names=db.get_usable_table_names()
)

system_message = SystemMessage(content=system)

tools.append(retriever_tool)

agent = create_react_agent(llm, tools, messages_modifier=system_message)

In [ ]:
for s in agent.stream(
    {"messages": [HumanMessage(content="What is the ...?")]}
):
    print(s)
    print("----")

In [ ]:
examples = [
  {
    "input": "What is the ...?",
    "query": ""
  }
]

In [ ]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate.from_template("User input: {input}\nSQL query: {query}")
prompt = FewShotPromptTemplate(
    examples=examples[:5],
    example_prompt=example_prompt,
    prefix="You are a PostgreSQL expert. Given an input question, create a syntactically correct PostgreSQL query to run. Unless otherwise specificed, do not return more than {top_k} rows.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries.",
    suffix="User input: {input}\nSQL query: ",
    input_variables=["input", "top_k", "table_info"],
)

In [ ]:
print(prompt.format(input="How many artists are there?", top_k=3, table_info="foo"))